In [1]:
# Imports
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection
from tempfile import TemporaryDirectory
import seaborn as sns
import time
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(1991)
torch.manual_seed(1991)
torch.cuda.manual_seed(1991)

import random
random.seed(1991)

import numpy as np
np.random.seed(1991)

Running on device: CUDA


In [2]:

# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

100%|██████████| 170498071/170498071 [00:13<00:00, 12980334.05it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
from torch.optim.lr_scheduler import CosineAnnealingLR,CosineAnnealingWarmRestarts,StepLR
def kl_loss_sym(x,y):
    kl_loss = nn.KLDivLoss(reduction='batchmean')
    return kl_loss(nn.LogSoftmax(dim=-1)(x),y)

def unlearning(
        net,
        retain_loader,
        forget_loader,
        val_loader,
):
    """Simple unlearning by finetuning."""
    print('-----------------------------------')
    epochs = 8
    retain_bs = 256
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.005,
                          momentum=0.9, weight_decay=0)
    optimizer_retain = optim.SGD(net.parameters(), lr=0.001*retain_bs/64, momentum=0.9, weight_decay=1e-2)
    ##the learning rate is associated with the batchsize we used
    optimizer_forget = optim.SGD(net.parameters(), lr=3e-4, momentum=0.9, weight_decay=0)
    total_step = int(len(forget_loader)*epochs)
    retain_ld = DataLoader(retain_loader.dataset, batch_size=retain_bs, shuffle=True)
    retain_ld4fgt = DataLoader(retain_loader.dataset, batch_size=256, shuffle=True)
    scheduler = CosineAnnealingLR(optimizer_forget, T_max=total_step, eta_min=1e-6)

    net.train()
    for sample in forget_loader: ##First Stage
        inputs, output = sample
        inputs = inputs.to(DEVICE)
        optimizer.zero_grad()
        outputs = net(inputs)
        uniform_label = torch.ones_like(outputs).to(DEVICE) / outputs.shape[1] ##uniform pseudo label
        loss = kl_loss_sym(outputs, uniform_label) ##optimize the distance between logits and pseudo labels
        loss.backward()
        optimizer.step()

    net.train()
    for ep in range(epochs): ##Second Stage
        net.train()
        for sample_forget, sample_retain in zip(forget_loader, retain_ld4fgt):##Forget Round
            t = 1.15 ##temperature coefficient
            inputs_forget,inputs_retain = sample_forget[0],sample_retain[0]
            inputs_forget, inputs_retain = inputs_forget.to(DEVICE), inputs_retain.to(DEVICE)
            optimizer_forget.zero_grad()
            outputs_forget,outputs_retain = net(inputs_forget),net(inputs_retain).detach()
            loss = (-1 * nn.LogSoftmax(dim=-1)(outputs_forget @ outputs_retain.T/t)).mean() ##Contrastive Learning loss
            loss.backward()
            optimizer_forget.step()
            scheduler.step()
        for sample in retain_ld: ##Retain Round
            inputs, labels = sample
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer_retain.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer_retain.step()

    print('-----------------------------------')
    return net

In [4]:
from sklearn.linear_model import LogisticRegression

def train_mia_on_class(model, class_index, train_set, test_set, no_class=True, splits_num=5):
  model = model.to("cpu")

  if not no_class:
    # Select indencies where class is class_index
    train_class_set = np.where(np.array(train_set.targets) == class_index)[0]
    test_class_set = np.where(np.array(test_set.targets) == class_index)[0]
  else:
    train_class_set = np.arange(len(train_set))
    test_class_set = np.arange(len(test_set))

  max_len = min([len(train_class_set), len(test_class_set)])
  # Make equal sizes
  train_class_set = train_class_set[:max_len]
  test_class_set = test_class_set[:max_len]

  # Obtain subsets
  train_class_set = torch.utils.data.Subset(train_set, train_class_set)
  test_class_set = torch.utils.data.Subset(test_set, test_class_set)


  # Make them
  class_test_loader = torch.utils.data.DataLoader(
    test_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  class_train_loader = torch.utils.data.DataLoader(
    train_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  criterion = nn.CrossEntropyLoss(reduction="none")
  # Obtain train and test logits
  logits_train = []
  for i, (images, labels) in enumerate(class_train_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_train.append(np.concatenate((logits.detach().numpy()[0], losses)))

  logits_test = []
  for i, (images, labels) in enumerate(class_test_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_test.append((np.concatenate((logits.detach().numpy()[0], losses))))

  logits_train, logits_test = np.array(logits_train), np.array(logits_test)
  # Create dataset
  ys = [1] * max_len + [0] * max_len
  ys = np.array(ys)
  p = np.random.permutation(len(ys))
  logits = np.concatenate((logits_train, logits_test))

  logits = logits[p]
  ys = ys[p]

  # Fit logitstic regression
  clf = LogisticRegression(random_state=0, max_iter=1000)
  cv = model_selection.StratifiedShuffleSplit(
        n_splits=splits_num, random_state=0
    )

  clf = model_selection.cross_val_score(
        clf, logits, ys, cv=cv, scoring="accuracy"
    )

  # Output score
  # print(f"Mean MIA attack score for class {class_index} is: " + str(clf.mean()))
  return clf.mean()

In [5]:


# Check accuracy

def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    for i, (inputs, targets) in enumerate(loader, 0):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        outputs = net(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return correct / total

In [6]:

def compute_losses(net, loader):
    """Auxiliary function to compute per-sample losses"""

    criterion = nn.CrossEntropyLoss(reduction="none")
    all_losses = []

    for inputs, targets in loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        logits = net(inputs)
        losses = criterion(logits, targets).numpy(force=True)
        for l in losses:
            all_losses.append(l)

    return np.array(all_losses)

## Plain network without regularization
## 5 %

In [9]:


print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Baseline"
model_params = "cifar10_resnet18_baseline.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_ft_forget.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)

----- 5 PERCENT -------
Model name: Resnet18 Baseline
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 95.6
MIA score before on forget set: 0.9


-----------------------------------
-----------------------------------
Retain set accuracy FORGET model: 99.6%
Test set accuracy FORGET model: 74.5%
Forget set accuracy FORGET model: 13.2%

The MIA has an accuracy of 0.948 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 97.6
MIA score before on forget set: 0.96


-----------------------------------
-----------------------------------
Retain set accuracy FORGET model: 99.6%
Test set accuracy FORGET model: 75.4%
Forget set accuracy FORGET model: 12.8%

The MIA has an accuracy of 0.964 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 92.80000000000001
MIA score before on forget set: 0.91


-----------

In [10]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)


[0.9, 0.96, 0.91, 0.88, 0.9099999999999999, 0.87, 0.94, 0.87, 0.9, 0.9099999999999999]
[0.9480000000000001, 0.9640000000000001, 0.9119999999999999, 0.876, 0.9640000000000001, 0.9040000000000001, 0.9279999999999999, 0.924, 0.908, 0.932]
[95.6, 97.6, 92.80000000000001, 84.8, 94.8, 89.60000000000001, 97.6, 96.8, 97.2, 96.8]
[99.56792604501608, 99.64025162288723, 99.46940006029544, 99.5598255381585, 99.67037163588125, 99.59603673854936, 99.51563630516922, 99.75880366618426, 99.71459581139204, 99.59407591986013]
[13.200000000000001, 12.8, 11.600000000000001, 10.8, 9.6, 10.0, 10.0, 9.6, 13.200000000000001, 13.600000000000001]
[74.62, 75.16000000000001, 75.27000000000001, 75.22999999999999, 75.33999999999999, 74.92, 75.33, 75.14, 75.19, 75.24]


In [11]:

mean_mia_score_before_baseline = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_baseline = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_baseline = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_baseline = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_baseline = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_baseline = np.mean(np.array(Accuracy_test))

In [12]:


print(mean_mia_score_before_baseline)
print(mean_mia_score_after_baseline)

print(mean_accuracy_forget_before_baseline)

print(mean_accuracy_forget_after_baseline)
print(mean_accuracy_retain_baseline)
print(mean_accuracy_test_baseline)

0.905
0.9259999999999999
94.35999999999999
11.440000000000001
99.60869233433937
75.144


## Dropout model

In [13]:
def modify_model(model_train_reg):
  from collections import OrderedDict

  layer1 = model_train_reg.layer1

  for indx in range(len(layer1)):
    modified_layers = OrderedDict()
    for name, feature in layer1[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer1[indx] = nn.Sequential(modified_layers)



  layer2 = model_train_reg.layer2

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer2[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer2[indx] = nn.Sequential(modified_layers)

  layer3 = model_train_reg.layer3

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer3[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)
    model_train_reg.layer3[indx] = nn.Sequential(modified_layers)

  layer4 = model_train_reg.layer4

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer4[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer4[indx] = nn.Sequential(modified_layers)

  return model_train_reg

In [14]:


print("----- 5 PERCENT DROPOUT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Dropout"
model_params = "cifar10_resnet18_l2_dropout_regularization.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft = model_forget_ft.to(DEVICE)
  model_forget_ft = modify_model(model_forget_ft)

  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_ft_forget.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)

----- 5 PERCENT DROPOUT -------
Model name: Resnet18 Dropout
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 84.0
MIA score before on forget set: 0.85


-----------------------------------
-----------------------------------
Retain set accuracy FORGET model: 79.7%
Test set accuracy FORGET model: 74.4%
Forget set accuracy FORGET model: 15.6%

The MIA has an accuracy of 0.904 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 90.0
MIA score before on forget set: 0.93


-----------------------------------
-----------------------------------
Retain set accuracy FORGET model: 80.2%
Test set accuracy FORGET model: 74.5%
Forget set accuracy FORGET model: 15.6%

The MIA has an accuracy of 0.908 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 78.0
MIA score before on forget set: 0.85


----------------

In [15]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.85, 0.9299999999999999, 0.85, 0.86, 0.8999999999999999, 0.84, 0.9, 0.94, 0.8600000000000001, 0.89]
[0.9039999999999999, 0.908, 0.8960000000000001, 0.876, 0.908, 0.8800000000000001, 0.8480000000000001, 0.916, 0.9119999999999999, 0.9199999999999999]
[84.0, 90.0, 78.0, 61.199999999999996, 79.60000000000001, 70.0, 80.80000000000001, 85.6, 87.2, 88.8]
[79.62497739011596, 80.18488745980707, 80.12821801081212, 79.33716536699092, 79.37854242874944, 79.74876896794292, 79.69811472444427, 79.79781739252768, 80.15515716697483, 79.73350483349077]
[13.200000000000001, 15.6, 10.8, 11.600000000000001, 11.600000000000001, 12.4, 13.600000000000001, 17.2, 15.2, 13.600000000000001]
[73.89, 74.63, 74.33, 73.57000000000001, 74.11, 74.16, 74.31, 74.33999999999999, 74.44, 74.11]


In [16]:


mean_mia_score_before_dropout = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_dropout = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_dropout = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_dropout = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_dropout = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_dropout = np.mean(np.array(Accuracy_test))

In [17]:


print(mean_mia_score_before_dropout)
print(mean_mia_score_after_dropout)

print(mean_accuracy_forget_before_dropout)

print(mean_accuracy_forget_after_dropout)
print(mean_accuracy_retain_dropout)
print(mean_accuracy_test_dropout)


0.882
0.8968000000000002
80.52000000000001
13.48
79.77871537418561
74.189


## Augmentation model

In [18]:

# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
         transforms.RandomRotation(30), # Randomly rotate some images by 20 degrees
         transforms.RandomHorizontalFlip(), # Randomly horizontal flip the images
         transforms.ColorJitter(brightness = 0.1, # Randomly adjust color jitter of the images
                                                            contrast = 0.1,
                                                            saturation = 0.1),
         transforms.RandomAdjustSharpness(sharpness_factor = 2,
                                                                      p = 0.1), # Randomly adjust sharpness
          transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
        transforms.ToTensor(),
    ]
)

to_tensor_transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)


# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=to_tensor_transform
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [19]:


print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Augmentation"
model_params = "cifar10_resnet18_augmentation.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_forget_ft.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)


----- 5 PERCENT -------
Model name: Resnet18 Augmentation
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 78.8
MIA score before on forget set: 0.92


-----------------------------------
-----------------------------------
Retain set accuracy FORGET model: 77.6%
Test set accuracy FORGET model: 77.0%
Forget set accuracy FORGET model: 16.8%

The MIA has an accuracy of 0.884 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 80.4
MIA score before on forget set: 0.9


-----------------------------------
-----------------------------------
Retain set accuracy FORGET model: 77.1%
Test set accuracy FORGET model: 76.7%
Forget set accuracy FORGET model: 12.4%

The MIA has an accuracy of 0.912 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 75.2
MIA score before on forget set: 0.88


--------------------

In [20]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.92, 0.8999999999999999, 0.88, 0.8, 0.89, 0.87, 0.8799999999999999, 0.89, 0.89, 0.77]
[0.884, 0.9120000000000001, 0.8560000000000001, 0.8800000000000001, 0.8800000000000001, 0.876, 0.9, 0.8480000000000001, 0.9200000000000002, 0.908]
[78.8, 80.4, 75.2, 66.0, 72.8, 71.6, 84.8, 82.0, 86.8, 80.4]
[77.4570888772762, 77.13952647023355, 77.65831306899254, 77.89367902723345, 77.52029905940992, 77.72709003215435, 77.69560061900837, 77.31594074729162, 77.19671999356862, 77.85951160687368]
[16.400000000000002, 14.000000000000002, 12.8, 10.8, 13.600000000000001, 11.600000000000001, 11.600000000000001, 15.2, 10.8, 11.200000000000001]
[77.19, 76.55999999999999, 77.03999999999999, 76.5, 76.73, 77.16, 77.03999999999999, 76.89, 76.39, 77.14]


In [21]:


mean_mia_score_before_aug = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_aug = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_aug = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_aug = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_aug = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_aug = np.mean(np.array(Accuracy_test))

In [22]:


print(mean_mia_score_before_aug)
print(mean_mia_score_after_aug)

print(mean_accuracy_forget_before_aug)

print(mean_accuracy_forget_after_aug)
print(mean_accuracy_retain_aug)
print(mean_accuracy_test_aug)


0.869
0.8864000000000001
77.87999999999998
12.8
77.54637695020423
76.86399999999999
